<a href="https://colab.research.google.com/github/KaifAhmad1/code-test/blob/main/Face_Similarity_Matching.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Reverse Image Search System for Defensive Forensics**

In [1]:
!pip install -q torch transformers langchain langgraph numpy pillow requests vllm langchain_community groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 93.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 67.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 78.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.4/151.4 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import torch
import numpy as np
import requests
from io import BytesIO
from PIL import Image, ImageEnhance, ImageFilter
import cv2
import matplotlib.pyplot as plt
import networkx as nx

from transformers import CLIPProcessor, CLIPModel
from vllm import LLMEngine
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# Use the new import statement for LangGraph
from langgraph.graph import Graph, StateGraph, END

In [3]:
# Load CLIP model and processor
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/605M [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

In [4]:
######################################
# 2. ADVANCED IMAGE PREPROCESSING FUNCTIONS
######################################
def preprocess_image(image_url: str) -> Image.Image:
    """
    Download and preprocess the image.
    Enhances contrast, sharpens, and returns a clean RGB image.
    """
    response = requests.get(image_url)
    image = Image.open(BytesIO(response.content)).convert("RGB")
    enhancer = ImageEnhance.Contrast(image)
    image = enhancer.enhance(1.5)
    image = image.filter(ImageFilter.SHARPEN)
    return image

In [5]:
######################################
# 3. MULTI-MODAL EMBEDDING FUNCTIONS
######################################
def generate_clip_embedding_vllm(image: Image.Image) -> np.ndarray:
    """
    Generate a robust image embedding using the CLIP model accelerated via vLLM.
    """
    inputs = clip_processor(images=image, return_tensors="pt")
    with torch.no_grad():
        # Use vLLM for fast inference.
        outputs = clip_engine.infer(inputs)
    embedding = outputs[0].numpy()
    norm = np.linalg.norm(embedding)
    return embedding / norm if norm != 0 else embedding

def extract_classical_features(image: Image.Image) -> np.ndarray:
    """
    Extract classical features using ORB.
    """
    image_np = np.array(image)
    gray = cv2.cvtColor(image_np, cv2.COLOR_RGB2GRAY)
    orb = cv2.ORB_create()
    keypoints, descriptors = orb.detectAndCompute(gray, None)
    return descriptors if descriptors is not None else np.array([])

In [6]:
######################################
# 4. SEARCH FUNCTIONS (Simulated API Calls)
######################################
def search_private_db(embedding: np.ndarray) -> list:
    return [{"source": "Private DB", "match": "Person_123", "score": 0.91}]

def search_twitter(embedding: np.ndarray) -> list:
    return [{"source": "Twitter", "match": "Tweet_Image_456", "score": 0.87}]

def search_reddit(embedding: np.ndarray) -> list:
    return [{"source": "Reddit", "match": "Reddit_Post_321", "score": 0.89}]

def search_instagram(embedding: np.ndarray) -> list:
    return [{"source": "Instagram", "match": "Insta_Post_654", "score": 0.88}]

def search_osint_sources(embedding: np.ndarray) -> list:
    return [{"source": "OSINT", "match": "DarkWeb_Post_999", "score": 0.83}]

def merge_search_results(*results: list) -> list:
    merged = []
    for result in results:
        merged.extend(result)
    return merged

In [7]:
######################################
# 5. DECISION FUSION WITH ADVANCED LLM
######################################
def decision_fusion(results: list) -> str:
    prompt = PromptTemplate(
        input_variables=["results"],
        template=(
            "The following reverse image search results were retrieved from multiple sources:\n"
            "{results}\n\n"
            "Provide a detailed forensic assessment considering:\n"
            "1. Cross-referencing of entities across sources.\n"
            "2. Temporal and geographic correlations if available.\n"
            "3. Reliability of each source and any discrepancies observed.\n\n"
            "Summarize your findings in two to three sentences."
        )
    )
    chain = LLMChain(llm=OpenAI(temperature=0), prompt=prompt)
    response = chain.run({"results": str(results)})
    return response.strip()

In [8]:
######################################
# 6. GRAPH & LINK ANALYSIS FOR VISUALIZATION
######################################
def graph_link_analysis(results: list) -> None:
    G = nx.Graph()
    for result in results:
        source = result['source']
        match = result['match']
        score = result['score']
        G.add_edge(source, match, weight=score)
    pos = nx.spring_layout(G)
    plt.figure(figsize=(8, 6))
    nx.draw(G, pos, with_labels=True, node_color='lightblue', edge_color='gray')
    plt.title("Graph & Link Analysis of Search Results")
    plt.show()

In [9]:
######################################
# 7. BUILD ADVANCED PIPELINE USING STATEGRAPH
######################################
def build_advanced_pipeline(image_url: str) -> str:
    """
    Constructs a pipeline using StateGraph with the following steps:
      - Preprocess Image
      - Generate CLIP Embedding via vLLM
      - Extract Classical Features (for hybrid search)
      - Query multiple sources (Private DB, Twitter, Reddit, Instagram, OSINT)
      - Merge search results
      - Decision fusion via LLM for forensic assessment
      - Graph analysis (optional visualization)
    """
    sg = StateGraph()

    # Define states using functions.
    sg.add_state("Preprocess", lambda _: preprocess_image(image_url))
    sg.add_state("Embed", lambda img: generate_clip_embedding_vllm(img))
    sg.add_state("Classical", lambda img: extract_classical_features(img))

    # Search states (using the embedding)
    sg.add_state("Search_Private", lambda emb: search_private_db(emb))
    sg.add_state("Search_Twitter", lambda emb: search_twitter(emb))
    sg.add_state("Search_Reddit", lambda emb: search_reddit(emb))
    sg.add_state("Search_Instagram", lambda emb: search_instagram(emb))
    sg.add_state("Search_OSINT", lambda emb: search_osint_sources(emb))

    # Merge state: combine all search results.
    sg.add_state("MergeResults", lambda priv, tw, red, insta, osint: merge_search_results(priv, tw, red, insta, osint))

    # Decision fusion state.
    sg.add_state("DecisionFusion", lambda merged: decision_fusion(merged))

    # Define transitions.
    sg.add_transition("Preprocess", "Embed")
    sg.add_transition("Embed", "Search_Private")
    sg.add_transition("Embed", "Search_Twitter")
    sg.add_transition("Embed", "Search_Reddit")
    sg.add_transition("Embed", "Search_Instagram")
    sg.add_transition("Embed", "Search_OSINT")

    sg.add_transition("Search_Private", "MergeResults")
    sg.add_transition("Search_Twitter", "MergeResults")
    sg.add_transition("Search_Reddit", "MergeResults")
    sg.add_transition("Search_Instagram", "MergeResults")
    sg.add_transition("Search_OSINT", "MergeResults")

    sg.add_transition("MergeResults", "DecisionFusion")
    sg.add_transition("DecisionFusion", END)
    sg.add_state(END, lambda result: result)

    # Execute the state graph.
    final_assessment = sg.run(input_data=None)

    # Optionally, perform graph analysis for visualization using merged results.
    merged_results = merge_search_results(
        search_private_db(np.array([0])),  # dummy input for demonstration
        search_twitter(np.array([0])),
        search_reddit(np.array([0])),
        search_instagram(np.array([0])),
        search_osint_sources(np.array([0]))
    )
    graph_link_analysis(merged_results)

    return final_assessment

In [10]:
######################################
# 8. EXECUTE THE ADVANCED PIPELINE
######################################
if __name__ == "__main__":
    test_image_url = "https://example.com/test_image.jpg"  # Replace with a valid image URL.
    final_result = build_advanced_pipeline(test_image_url)

    print("Final Forensic Assessment:")
    print(final_result)

ValueError: Must provide state_schema or input and output